# Project 2

## Topic : Stereo reconstruction and Non-linear optimization

#### Instructions
<ul>
    <li> The second project of the course is designed to get you familiar with stereo reconstruction, and non-linear optimization </li>
    <li> Use python for this project. PILLOW and OpenCV are permitted for image I/O. </li>
    <li> Submit this notebook as a zipped file on moodle. The format should be $<$team_id$>$_$<$team_ name$>$.zip. Both members have to submit this zip file. </li>
    <li> A seperate report is not needed if you're coding in the notebook itself. Please provide adequate descriptions of the approaches you've taken. Also mention work distribution for the two members. </li>
    <li> Refer to the late day policy. Start early </li> 
    <li> Download data from here: https://iiitaphyd-my.sharepoint.com/:f:/g/personal/aryan_sakaria_students_iiit_ac_in/Er5C7351IAlFsvwHUesFeSQBQtlSiAS7AORSEJT2qH_8_w?e=ol98k9  </li>
</ul>

----
### PART 1: Stereo dense reconstruction

3-D point clouds are very useful in robotics for several tasks such as object detection, motion estimation (3D-3D matching or 3D-2D matching), SLAM, and other forms of scene understanding.  Stereo camerasprovide  us  with  a  convenient  way  to  generate  dense  point  clouds.Densehere,  in  contrast  tosparse,means all the image points are used for the reconstruction.  In this part of the assignment you will begenerating a dense 3D point cloud reconstruction of a scene from stereo images.

#### Procedure: 

<ol>
    <li> Generate a disparity map for each stereo pair.  Use OpenCV (e.g.  StereoSGBM) for this.  Notethat the images provided are already rectified and undistorted. </li>
    <li> Then, using the camera parameters and baseline information generate colored point clouds fromeach disparity map.  Some points will have invalid disparity values, so ignore them.  Use [Open3D]for storing your point clouds. </li>
    <li> Register (or transform) all the generated point clouds into your world frame by using the providedground truth poses. </li>
    <li> Visualize the registered point cloud data, in color.  Use Open3D for this </li>
</ol>
    

In [17]:
#import libraries:
import numpy as np 
from sklearn.preprocessing import normalize #normalizing gives better results. Experiment with this
import cv2
import open3d as o3d

In [18]:
def read_transformations(filename='./mr2020_project_data/poses.txt'):
    f = open(filename, 'r')
    lines = f.readlines()
    transformation_list = []
    for i in range(len(lines)):
        transformation_list_temp = lines[i].split()
        temp_rot = [] 
        temp_rot.append( (transformation_list_temp[0:4] ) ) 
        temp_rot.append( (transformation_list_temp[4:8]  ) ) 
        temp_rot.append( (transformation_list_temp[8:12]  ) ) 
        transformation_list.append(temp_rot)
    return transformation_list
tr_list=read_transformations()

Provide explanation in this cell: 


In [19]:
#your code here
def convert_str_to_int(tr_list):
    tr_mat_list=[]
    for i in range(0,len(tr_list)):
        tr_mat_str=tr_list[i]
        tr_mat=np.zeros((3,4),float)
        for j in range(0,3):
            for k in range(0,4):
                tr_mat[j][k]=float(tr_mat_str[j][k])
        tr_mat_list.append(tr_mat)
    return tr_mat_list
tr_mat_list=convert_str_to_int(tr_list)


K=np.array([[707.0912,0,601.8873],
           [0,707.0912,183.1104],
           [0,0,1]])
f_l=K[0][0]
b=0.53790448812
Q = np.float32(    [[ 1,  0,  0, -613],
                    [ 0, -1,  0,  185],
                    [ 0,  0,  0,  f_l],
                    [ 0,  0,  1/b, 0]])

def get_cam_pts_frm_Q(Q,disparity_pts):
    points = []
    for dis in disparity_pts:
        point = Q@(dis)
        point=point/point[3]
        points.append(point)
    return np.array(points)    

In [20]:
f=K[0][0]
pcd_gl=o3d.geometry.PointCloud()
pcd = o3d.geometry.PointCloud()
pcd_list=[]
for i in range(60,81):
    img_1 = cv2.imread('./mr2020_project_data/img2/00000004'+str(i)+'.png')
    img_2 = cv2.imread('./mr2020_project_data/img3/00000004'+str(i)+'.png')
    
    win_size = 5
    blockSize= 5
    min_disp = -39
    num_disp = 144 
    
    stereo = cv2.StereoSGBM_create(minDisparity= min_disp,
           numDisparities = num_disp,
           blockSize = 5,
           uniquenessRatio = 10,
           speckleWindowSize = 150,
           speckleRange = 32,
           disp12MaxDiff = 1,
           P1 = 8*3*blockSize**2,
           P2 =32*3*blockSize**2,
           preFilterCap=63) 
        
    disparity= stereo.compute(img_1, img_2).astype(np.float32) / 64
    disparity = (disparity-min_disp)/num_disp
    Dim = disparity.shape
    
    disparity_pts=[]
    for p in range(Dim[0]):
        for q in range(Dim[1]):
            disparity_pts.append([q,p,disparity[p,q],1])
    disparity_pts = np.array(disparity_pts)
        
    cam_pts=get_cam_pts_frm_Q(Q,disparity_pts)
    cam_pts=cam_pts.T
        
    colors = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
    mask = disparity >= disparity.min()
    colors = colors[mask]
    colors=colors/255
    world_pts=(tr_mat_list[i-60])@cam_pts
    world_pts[0]=-world_pts[0]
        
    pcd.points = o3d.utility.Vector3dVector(world_pts.T)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    pcd_list.append(pcd)
    pcd_gl=pcd_gl+pcd
    
pts=np.asarray(pcd_gl.points)
clrs=np.asarray(pcd_gl.colors)
mask = ((-1500 <= pts[:,1]) & (pts[:,1] < 1500) &
         (-1500 <= pts[:,2]) & (pts[:,2] < 1500) &
        (-1500 <= pts[:,0]) & (pts[:,0] < 1500))
pts=pts[mask]
print(np.shape(pts))
clrs=clrs[mask]
pcd_gl.points=o3d.utility.Vector3dVector(pts)
pcd_gl.colors=o3d.utility.Vector3dVector(clrs)

o3d.visualization.draw_geometries([pcd_gl])

(7021725, 3)


----
### PART 2: Motion estimation using iterative PnP

Using the generated reconstruction from the previous part, synthesize a new image taken by a virtualmonocular camera fixed at any arbitrary position and orientation.  Your task in this part is to recoverthis pose using an iterative Perspective-from-n-Points (PnP) algorithm. 

#### Procedure: 

<ol>
    <li> Obtain a set of 2D-3D correspondences between the the image and the point cloud.  Since hereyou’re generating the image, this should be easy to obtain. </li>
    <li> For this set of correspondences compute the total reprojection error c= $\sum_{i} ‖x_i−P_{k}X_i‖^2 $    where $P_{k}= K[R_{k}|t_{k}]$, $X_{i}$ is the 3D point in the world frame, $x_{i}$ is its corresponding projection. </li>
    <li> Solve for the pose $T_{k}$ that minimizes this non-linear reprojection error using a Gauss-Newton (GN)scheme.  Recall that in GN we start with some initial estimated value $x_{o}$ and iteratively refine the estimate using $x_{1}$= $∆x+x_0$, where $∆x$ is obtained by solving the normal equations $J^{T}J∆x$= -$J^{T}e$, until convergence.The main steps in this scheme are computing the corresponding Jacobians and updating the estimates correctly.  For our problem,  use a 12×1 vector parameterization for $T_{k}$(the top 3×4submatrix).  Run the optimization for different choices of initialization and report your observations. </li>
</ol>

In [19]:
o3d.visualization.draw_geometries([pcd_1])
#print(np.shape(pts_3d))
pts_3d1=np.asarray(pcd_1.points).T
def append_ones(pts_3d1):
    pts_3d1=np.append(pts_3d1,np.ones((1,np.shape(pts_3d1)[1]),float),axis=0)
    return pts_3d1

pts_3d=append_ones(pts_3d1)
P_gt=K@tr_mat_list[0]
print(np.shape(P_gt))
print(np.shape(pts_3d))
pixel_gt=P_gt@pts_3d
pixel_gt=pixel_gt/pixel_gt[2]
print(np.shape(pixel_gt))
r=np.empty((0,1),float)
def calc_residual(p,pixel_gt,pts_3d):
    #r=np.empty((0,1),float)
    r=[]
    for i in range(0,np.shape(pts_3d)[1]):
        r_pt_x=[(p[0][0]*pts_3d[0][i]+p[1][0]*pts_3d[1][i]+p[2][0]*pts_3d[2][i]+p[3][0])/(p[8][0]*pts_3d[0][i]+p[9][0]*pts_3d[1][i]+p[10][0]*pts_3d[2][i]+p[11][0])-pixel_gt[0][i]]
        r_pt_y=[(p[4][0]*pts_3d[0][i]+p[5][0]*pts_3d[1][i]+p[6][0]*pts_3d[2][i]+p[7][0])/(p[8][0]*pts_3d[0][i]+p[9][0]*pts_3d[1][i]+p[10][0]*pts_3d[2][i]+p[11][0])-pixel_gt[1][i]]
        #r_pt_z=[p[8][0]*pts_3d[0][i]+p[9][0]*pts_3d[1][i]+p[10][0]*pts_3d[2][i]+p[11][0]-pixel_gt[2][i]]
        r.append(r_pt_x)
        r.append(r_pt_y)
        #r.append(r_pt_z)
        #r=np.append(r,r_pt,axis=0)
                         
    return np.array(r)
#r=calc_residual((P_gt.flatten()).reshape(12,1),pixel_gt,pts_3d)
#print(r)
def calc_jacobian(pts_3d,p):
    img_pts=(p.reshape(3,4))@pts_3d
    J=[]
    for i in range(0,np.shape(pts_3d)[1]):
        J_x=[pts_3d[0][i]/img_pts[2][i],pts_3d[1][i]/img_pts[2][i],pts_3d[2][i]/img_pts[2][i],1/img_pts[2][i],0,0,0,0,-pts_3d[0][i]*img_pts[0][i]/(img_pts[2][i]**2),-pts_3d[1][i]*img_pts[0][i]/(img_pts[2][i]**2),-pts_3d[2][i]*img_pts[0][i]/(img_pts[2][i]**2),-img_pts[0][i]/(img_pts[2][i]**2)]
        J_y=[0,0,0,0,pts_3d[0][i]/img_pts[2][i],pts_3d[1][i]/img_pts[2][i],pts_3d[2][i]/img_pts[2][i],1/img_pts[2][i],0,0,0,0]
        J_z=[0,0,0,0,0,0,0,0,pts_3d[0][i],pts_3d[1][i],pts_3d[2][i],1]
        J.append(J_x)
        J.append(J_y)
        #J.append(J_z)
    return np.array(J)
#J=calc_jacobian(pts_3d)
#print(np.shape(J))

(3, 4)
(4, 453620)
(3, 453620)


In [20]:
def check_conv(p,p1,threshold):
    l=np.shape(p)[0]
    flag=bool(1)
    for i in range(0,l):
        flag=(flag and (abs(p[i][0]-p1[i][0])<threshold))
    return flag

def gauss_newton(p_init,pts_3d,pixel_gt,threshold,maxitr) :
    itr=0
    p=p_init
    while(itr<maxitr):
        J=calc_jacobian(pts_3d,p)
        r=calc_residual(p,pixel_gt,pts_3d)
        #print(r)
        step=np.linalg.inv((J.T@J))@(J.T@r)
        #print(p)
        p1=p-step
        #print(p1)
        flag=check_conv(p,p1,threshold)
        if flag:
            print(itr)
            return p1
        p=p1
        itr=itr+1
    return p
p_init1=(P_gt.flatten()).reshape(12,1)+np.random.normal(0,100,(12,1))
P1=K@np.array([[1,0,0,135],
              [0,1,0,248],
              [0,0,1,1000]])
p_init2=(P1.flatten()).reshape(12,1)
p_init3=np.zeros((12,1),float)
print(p_init3)
p_opt=gauss_newton(p_init3,pts_3d,pixel_gt,1e-9,1e4)
print(p_opt)
print((P_gt.flatten()).reshape(12,1))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


<ipython-input-19-0037190fd27c>:35: RuntimeWarning: divide by zero encountered in double_scalars
  J_x=[pts_3d[0][i]/img_pts[2][i],pts_3d[1][i]/img_pts[2][i],pts_3d[2][i]/img_pts[2][i],1/img_pts[2][i],0,0,0,0,-pts_3d[0][i]*img_pts[0][i]/(img_pts[2][i]**2),-pts_3d[1][i]*img_pts[0][i]/(img_pts[2][i]**2),-pts_3d[2][i]*img_pts[0][i]/(img_pts[2][i]**2),-img_pts[0][i]/(img_pts[2][i]**2)]
<ipython-input-19-0037190fd27c>:35: RuntimeWarning: invalid value encountered in double_scalars
  J_x=[pts_3d[0][i]/img_pts[2][i],pts_3d[1][i]/img_pts[2][i],pts_3d[2][i]/img_pts[2][i],1/img_pts[2][i],0,0,0,0,-pts_3d[0][i]*img_pts[0][i]/(img_pts[2][i]**2),-pts_3d[1][i]*img_pts[0][i]/(img_pts[2][i]**2),-pts_3d[2][i]*img_pts[0][i]/(img_pts[2][i]**2),-img_pts[0][i]/(img_pts[2][i]**2)]
<ipython-input-19-0037190fd27c>:36: RuntimeWarning: divide by zero encountered in double_scalars
  J_y=[0,0,0,0,pts_3d[0][i]/img_pts[2][i],pts_3d[1][i]/img_pts[2][i],pts_3d[2][i]/img_pts[2][i],1/img_pts[2][i],0,0,0,0]
<ipython-inpu

KeyboardInterrupt: 